In [2]:
import pystac_client
import planetary_computer
import stackstac
import xarray as xr
import rioxarray as rio
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import warnings
from odc.stac import stac_load
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
# Geographic bounds for NYC (Bronx/Manhattan area)
lower_left  = (40.75, -74.01)  # (lat, lon)
upper_right = (40.88, -73.86)  # (lat, lon)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])  # (min_lon, min_lat, max_lon, max_lat)

# Some user-defined QA/classification masks:
LANDSAT_VALID_QA = [322, 386]    # simplistic "clear/water" pixel QA values
S2_VALID_SCL = [2, 4, 5, 6]      # SCL: dark area, vegetation, bare, water

In [4]:
def produce_landsat_tiff_v4(date_window, approach="single", out_filename="Landsat_LST_v4.tiff"):
    """
    Produce a Landsat LST GeoTIFF either using a single best scene (lowest cloud cover)
    or a median mosaic over the given date window.
    """
    print(f"Date window: {date_window}, Approach: {approach}, Output: {out_filename}")
    
    # 1. Search Landsat-8 Collection 2 L2
    stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = stac.search(
        bbox=bounds,
        datetime=date_window,
        collections=["landsat-c2-l2"],
        query={"platform": {"in": ["landsat-8"]}}
    )
    items = list(search.get_items())
    print(f"  Found Landsat items: {len(items)}")

    if not items:
        print("  No Landsat items found! Skipping.")
        return

    signed_items = [planetary_computer.sign(item) for item in items]
    bands_of_interest = ["lwir11", "qa_pixel"]
    resolution = 30 / 111320.0  # ~30m in degrees

    ds = stac_load(
        signed_items,
        bands=bands_of_interest,
        crs="EPSG:4326",
        resolution=resolution,
        chunks={"x": 2048, "y": 2048},
        bbox=bounds
    ).persist()

    # 2. Convert thermal DN => Celsius
    scale = 0.00341802
    offset = 149.0
    ds["lwir11"] = ds["lwir11"] * scale + offset - 273.15

    # 3. Basic QA mask
    mask = ds["qa_pixel"].isin(LANDSAT_VALID_QA)
    ds_masked = ds.where(mask)

    if approach == "single":
        # Pick single item with lowest cloud cover
        # Sort items by cloud cover
        items_sorted = sorted(items, key=lambda i: i.properties["eo:cloud_cover"])
        best_item = items_sorted[0]
        best_item_id = best_item.id
        print(f"  => Single-scene selected: {best_item_id}, cloud={best_item.properties['eo:cloud_cover']}")

        # We only want data from that item in ds_masked
        # We'll filter by matching item ID in 'time' coordinate if present
        # Each time slice in ds corresponds to an item
        # So we can find index of best_item
        item_ids = [itm.id for itm in signed_items]
        idx = item_ids.index(best_item_id)
        # Subset that single time
        ds_masked = ds_masked.isel(time=idx) if "time" in ds_masked.dims else ds_masked

    elif approach == "median":
        # Use all items in ds_masked, compute median over time
        print("  => Using median mosaic over all scenes in date range.")
        if "time" in ds_masked.dims:
            ds_masked = ds_masked.median(dim="time", skipna=True)

    else:
        print(f"  Unknown approach: {approach}, skipping.")
        return

    # If "time" dimension still remains (e.g. 1 item), pick first time slice
    if "time" in ds_masked.dims:
        ds_masked = ds_masked.isel(time=0)

    # 4. Extract final data array
    final_data = ds_masked["lwir11"]

    # 5. Save to GeoTIFF
    final_data.rio.write_crs("EPSG:4326", inplace=True)
    final_data.rio.to_raster(out_filename, compress="lzw")
    print(f"  => Saved {out_filename}")


def produce_s2_tiff_v4(date_window, approach="single", out_filename="S2_indices_v4.tiff"):
    """
    Produce a Sentinel-2 multi-band indices GeoTIFF (NDVI, NDBI, NDWI).
    Either single best scene or a median mosaic over date_window.
    """
    print(f"Date window: {date_window}, Approach: {approach}, Output: {out_filename}")

    # 1. Search Sentinel-2 L2A
    stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = stac.search(
        bbox=bounds,
        datetime=date_window,
        collections=["sentinel-2-l2a"],
        query={"eo:cloud_cover": {"lt": 90}}  # keep broad, we'll filter if single
    )
    items = list(search.get_items())
    print(f"  Found S2 items: {len(items)}")

    if not items:
        print("  No Sentinel-2 items found! Skipping.")
        return

    signed_items = [planetary_computer.sign(item) for item in items]
    bands_of_interest = ["B02", "B03", "B04", "B08", "B11", "SCL"]
    resolution = 10 / 111320.0  # 10m in degrees

    ds = stac_load(
        signed_items,
        bands=bands_of_interest,
        crs="EPSG:4326",
        resolution=resolution,
        chunks={"x": 2048, "y": 2048},
        bbox=bounds
    ).persist()

    ds = ds.astype(float)

    # 2. Basic SCL mask
    mask = ds["SCL"].isin(S2_VALID_SCL)
    ds_masked = ds.where(mask)

    if approach == "single":
        # Sort by cloud cover, pick best item
        items_sorted = sorted(items, key=lambda i: i.properties["eo:cloud_cover"])
        best_item = items_sorted[0]
        best_item_id = best_item.id
        print(f"  => Single-scene selected: {best_item_id}, cloud={best_item.properties['eo:cloud_cover']}")

        # Filter ds_masked to that item
        item_ids = [itm.id for itm in signed_items]
        idx = item_ids.index(best_item_id)
        if "time" in ds_masked.dims:
            ds_masked = ds_masked.isel(time=idx)

    elif approach == "median":
        # Median mosaic
        print("  => Using median mosaic over all S2 scenes in date range.")
        if "time" in ds_masked.dims:
            ds_masked = ds_masked.median(dim="time", skipna=True)

    else:
        print(f"  Unknown approach: {approach}, skipping.")
        return

    # If "time" dimension remains with size=1
    if "time" in ds_masked.dims:
        ds_masked = ds_masked.isel(time=0)

    # 3. Compute NDVI, NDBI, NDWI
    ndvi = (ds_masked["B08"] - ds_masked["B04"]) / (ds_masked["B08"] + ds_masked["B04"])
    ndbi = (ds_masked["B11"] - ds_masked["B08"]) / (ds_masked["B11"] + ds_masked["B08"])
    ndwi = (ds_masked["B03"] - ds_masked["B08"]) / (ds_masked["B03"] + ds_masked["B08"])

    # 4. Combine into a 3-band array
    indices = xr.concat([ndvi, ndbi, ndwi], dim="band")
    indices["band"] = [1, 2, 3]  # band 1=NDVI, 2=NDBI, 3=NDWI

    # 5. Save
    indices.rio.write_crs("EPSG:4326", inplace=True)
    indices.rio.to_raster(out_filename, compress="lzw")
    print(f"  => Saved {out_filename}")

In [5]:
if __name__ == "__main__":
    # -------------------------------------------------------------------
    # EXAMPLES: Generate multiple combos of Tiffs
    # You can tweak or add more. Then you'll see multiple Tiffs in your folder.
    # Try each pair in your main notebook.

    # 1) Landsat single, 3-month window
    produce_landsat_tiff_v4("2021-06-01/2021-09-01", "single", "Landsat_LST_v4_single_0601_0901.tiff")

    # 2) Landsat median, ~5-month window
    produce_landsat_tiff_v4("2021-05-01/2021-10-01", "median", "Landsat_LST_v4_median_0501_1001.tiff")

    # 3) Sentinel single, 3-month window
    produce_s2_tiff_v4("2021-06-01/2021-09-01", "single", "S2_indices_v4_single_0601_0901.tiff")

    # 4) Sentinel median, 5-month window
    produce_s2_tiff_v4("2021-05-01/2021-10-01", "median", "S2_indices_v4_median_0501_1001.tiff")

    # Optionally, produce more combos:
    # produce_landsat_tiff_v4("2021-07-15/2021-07-30", "median", "Landsat_LST_v4_median_15_30.tiff")
    # produce_s2_tiff_v4("2021-07-15/2021-07-30", "median", "S2_indices_v4_median_15_30.tiff")

    print("\nAll V4 TIFF generation done.")


=== produce_landsat_tiff_v4 ===
Date window: 2021-06-01/2021-09-01, Approach: single, Output: Landsat_LST_v4_single_0601_0901.tiff


C:\workspace\python\ai-challenge-2025\.venv\lib\site-packages\pystac_client\item_search.py:888: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


  Found Landsat items: 17
  => Single-scene selected: LC08_L2SP_014031_20210607_02_T1, cloud=6.88
  => Saved Landsat_LST_v4_single_0601_0901.tiff

=== produce_landsat_tiff_v4 ===
Date window: 2021-05-01/2021-10-01, Approach: median, Output: Landsat_LST_v4_median_0501_1001.tiff
  Found Landsat items: 29
  => Using median mosaic over all scenes in date range.
  => Saved Landsat_LST_v4_median_0501_1001.tiff

=== produce_s2_tiff_v4 ===
Date window: 2021-06-01/2021-09-01, Approach: single, Output: S2_indices_v4_single_0601_0901.tiff
  Found S2 items: 27
  => Single-scene selected: S2B_MSIL2A_20210606T153809_R011_T18TWL_20210609T213133, cloud=1.243344
  => Saved S2_indices_v4_single_0601_0901.tiff

=== produce_s2_tiff_v4 ===
Date window: 2021-05-01/2021-10-01, Approach: median, Output: S2_indices_v4_median_0501_1001.tiff
  Found S2 items: 45
  => Using median mosaic over all S2 scenes in date range.
  => Saved S2_indices_v4_median_0501_1001.tiff

All V4 TIFF generation done.
